In [2]:
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp

from astropy.coordinates import SkyCoord, Angle, EarthLocation, get_sun
from astropy.io import ascii
from astropy.table import join, Table, vstack
from astropy.time import Time
import astropy.units as u

from astroquery.utils.tap.core import TapPlus

import timeit

In [ ]:
Table_A2 = ascii.read("Table_A2.txt")

In [ ]:
upper_mu = 19.5
lower_mu = 18
mu_select = Table_A2[
    (Table_A2["Distance_modulus"] >= lower_mu) & 
    (Table_A2["Distance_modulus"] <= upper_mu)
]

## Circular cut based on coordinates from catalogue

### A-1: From intact catalogue

In [ ]:
lmc_center = SkyCoord(ra=80.894167*u.degree, dec=-69.756111*u.degree, frame='icrs')
radius = 5* u.degree

data_coords = SkyCoord(ra=Table_A2["RA"]*u.degree, dec=Table_A2["DEC"]*u.degree, frame='icrs')

# Compute the separation angular separation from LMC
sep = data_coords.separation(lmc_center)
circular_data = Table_A2[sep < radius]

### A-2: From $\mu$-filtered catalogue

In [ ]:
data_coords = SkyCoord(ra=mu_select["RA"]*u.degree, dec=mu_select["DEC"]*u.degree, frame='icrs')

sep = data_coords.separation(lmc_center)

mu_cut_data = mu_select[sep<radius]

## Comparism: ML and coordinate cuts

In [ ]:
mu18_ra  = mu_select["RA"] 
mu18_dec = mu_select["DEC"]

# ML_ra =  mu18_ra[cluster_0]
# ML_dec = mu18_dec[cluster_0]

CircularCut_ra  = circular_data["RA"]
CircularCut_dec = circular_data["DEC"]

CircularMu_ra  = mu_cut_data["RA"]
CircularMu_dec = mu_cut_data["DEC"]

In [ ]:
plt.figure(figsize=(8, 6))
# plt.scatter(mu18_ra, mu18_dec, s=10, color='gray', marker='o', alpha=0.3, label="Sources")
# plt.scatter(ML_ra, ML_dec, s=10, color='red', marker='o', alpha=0.3, label="Sources")
plt.scatter(CircularCut_ra, CircularCut_dec, s=10, color='blue', marker='o', alpha=0.3, label="Sources")
plt.scatter(CircularMu_ra, CircularMu_dec, s=10, color='green', marker='o', alpha=0.3, label="Sources")
plt.grid(True)

In [ ]:
print("DBSCAN sample:", len(ML_ra))
print("Circular cut sample:", len(CircularCut_ra))
print("Circular cut + mu sample:", len(CircularMu_ra))

## Export Table

In [ ]:
mu_cut_data.write('Muraveva_LMC-gaia_ID.csv', format='csv', overwrite=True)

## Query SkyMapper DR4

In [ ]:
gaia_id = ascii.read("Muraveva_LMC-gaia_ID.csv")
SkyMapper = TapPlus(url="https://api.skymapper.nci.org.au/public/tap/")

### Query `object_id`

In [ ]:
test_list = gaia_id[0:1000]

print("Begin query SMSS object ID")

id_table = Table(names=["object_id", "gaia_dr3_id1", "raj2000", "dej2000"], dtype=["int64", "int64", "float64", "float64"])
start = timeit.default_timer()

for (gaia_id, ra, dec) in zip(test_list["source_id"], test_list["RA"], test_list["DEC"]):
    #print(f"Querying GAIA id: {gaia_id}")
    prompt = f"""
    SELECT
        object_id, gaia_dr3_id1, raj2000, dej2000
        FROM
            dr4.master 
        WHERE 
            1=CONTAINS(POINT('ICRS', raj2000, dej2000),
                       CIRCLE('ICRS', {ra}, {dec}, 0.1 ))
            AND gaia_dr3_id1={gaia_id}
    """
    
    job = SkyMapper.launch_job(prompt)
    result = job.get_data()

    if len(result) == 1:
        #print("SMSS ID:",result['object_id'][0])
        id_table = vstack([id_table, result])
    #else:
        #print("! No result qureied from GAIA ID")
    #print("=========================")

stop = timeit.default_timer()
print("Done SMSS object ID query")
print('Time: ', stop - start)
print('Total', len(id_table), "SMSS ID queried from GAIA DR3 object ID")

### Query `object_id` (w/ `multiprocessing`)

In [ ]:
# Select first 5000 entries for querying
test_list = gaia_id[0:1000]

# Initialize an empty table to store results
id_table = Table(names=["object_id", "gaia_dr3_id1", "raj2000", "dej2000"], 
                 dtype=["int64", "int64", "float64", "float64"])

def query_skymapper(gaia_id, ra, dec):
    prompt = f"""
    SELECT
        object_id, gaia_dr3_id1, raj2000, dej2000
    FROM
        dr4.master 
    WHERE 
        1=CONTAINS(POINT('ICRS', raj2000, dej2000),
                   CIRCLE('ICRS', {ra}, {dec}, 0.1 ))
        AND gaia_dr3_id1={gaia_id}
    """
    try:
        job = SkyMapper.launch_job(prompt)
        result = job.get_data()
        if len(result) == 1:
            return result
    except Exception as e:
        print(f"Query failed for GAIA ID {gaia_id}: {e}")
    return None

print("Begin query SMSS object ID")
start = timeit.default_timer()

num_workers = mp.cpu_count()  # Use all available CPU cores
with mp.Pool(processes=num_workers) as pool:
    results = pool.starmap(query_skymapper, zip(test_list["source_id"], test_list["RA"], test_list["DEC"]))

# Filter out None results and merge tables
valid_results = [r for r in results if r is not None]
if valid_results:
    id_table = vstack(valid_results)

stop = timeit.default_timer()
print("Done SMSS object ID query")
print('Time:', stop - start)
print('Total', len(id_table), "SMSS ID queried from GAIA DR3 object ID")

# Save results to CSV
id_table.write("SMSS_results.csv", format="csv", overwrite=True)
print("Results saved to 'SMSS_results.csv'")

### Query `image_id`

In [ ]:
smss_id = ascii.read("Muraveva_LMC-smss_ID.csv")
test_list = smss_id[0:100]

In [ ]:
image_id = Table(names=["object_id", "image_id", "ra_img", "decl_img", "filter", "mag_psf", "e_mag_psf", "date", "exp_time", "flags"], dtype=["int64", "int64", "float64", "float64", "str", "float32", "float32", "float64", "float32", "int16"])

print("Begin query SMSS photometry")
start = timeit.default_timer()

for ID in test_list['object_id']:
    prompt = f"""
    SELECT
        object_id, image_id, ra_img, decl_img, f.filter, mag_psf, e_mag_psf, date, exp_time, flags
        FROM
            dr4.photometry f
        JOIN
            dr4.images USING (image_id)
        WHERE
            object_id={ID}
            AND f.filter IN ('g', 'r', 'i')
    """
    job = SkyMapper.launch_job(prompt)
    result = job.get_results()
    
    image_id = vstack([image_id, result])

stop = timeit.default_timer()
print("Done SMSS query")
print('Query time:', stop - start)

image_id.write('Muraveva_LMC-smss_photometry.csv', format='csv', overwrite=True)

### Query `image_id` (w/ `multiprocessing`)

In [ ]:
image_id = Table(names=["object_id", "image_id", "ra_img", "decl_img", "filter", "mag_psf", "e_mag_psf", "date", "exp_time", "flags"], 
                 dtype=["int64", "int64", "float64", "float64", "str", "float32", "float32", "float64", "float32", "int16"])

def query_photometry(object_id):
    prompt = f"""
    SELECT
        object_id, image_id, ra_img, decl_img, f.filter, mag_psf, e_mag_psf, date, exp_time, flags
    FROM
        dr4.photometry f
    JOIN
        dr4.images USING (image_id)
    WHERE
        object_id={object_id}
        AND f.filter IN ('g', 'r', 'i')
    """
    try:
        job = SkyMapper.launch_job(prompt)
        result = job.get_results()
        if len(result) > 0:
            return result  # Return results only if there's data
    except Exception as e:
        print(f"Query failed for object_id {object_id}: {e}")
    return None  # Return None if the query fails

print("Begin query SMSS photometry")
start = timeit.default_timer()

num_workers = mp.cpu_count()  # Use all available CPU cores
with mp.Pool(processes=num_workers) as pool:
    results = pool.map(query_photometry, test_list['object_id'])

# Filter out None results and merge tables
valid_results = [r for r in results if r is not None]
if valid_results:
    image_id = vstack(valid_results)

stop = timeit.default_timer()
print("Done SMSS query")
print('Query time:', stop - start)
print(f'Total {len(image_id)} photometric entries retrieved.')

## Light-curve-template fitting

### modify and match id from the files

In [2]:
import pandas as pd
import numpy as np
import apply_ugrizy_templates

# Load only required columns (if applicable) and limit rows for test cases
muraveva_df = pd.read_csv("Muraveva_LMC-gaia_ID.csv", usecols=["source_id", "RA", "DEC", "Type",
                                                               "Period", "[Fe/H]", "[Fe/H]_error", 
                                                               "E(B-V)", "E(B-V)_error", 
                                                               "Distance_modulus", "Distance_modulus_error"])
objid_df    = pd.read_csv("Muraveva_LMC-smss_ID.csv", usecols=["object_id", "gaia_dr3_id1"])
imageid_df  = pd.read_csv("Muraveva_LMC-smss_photometry.csv")

# Rename columns while reading (avoiding extra reassignment)
muraveva_df.rename(columns={'source_id': 'gaia_dr3_id1'}, inplace=True)

# Merge tables efficiently
merged_df = objid_df.merge(muraveva_df, on='gaia_dr3_id1', how="inner") \
                    .merge(imageid_df, on='object_id', how="inner")

### Using `apply_ugrizy_templates.py` (Braga+ 2024)

In [5]:
def mag_filter(image_ls):
    return len(image_ls) > 3  # Returns True if good, False if bad

def mjd_to_hjd(mjd, ra, dec, observatory='greenwich'):
    """
    Convert MJD to Heliocentric Julian Date (HJD).
    
    Parameters:
    - mjd: float, Modified Julian Date
    - ra: float, Right Ascension in degrees
    - dec: float, Declination in degrees
    - observatory: str, Name of the observing location (default: Greenwich)
    
    Returns:
    - HJD: float, Heliocentric Julian Date
    """
    # Convert MJD to JD
    time = Time(mjd, format='mjd', scale='utc')
    
    # Define the observatory location (default Greenwich, override if needed)
    location = EarthLocation.of_site(observatory)
    
    # Define the celestial target coordinates
    target = SkyCoord(ra=ra * u.deg, dec=dec * u.deg)
    
    # Get the Sun's position
    light_time_correction = time.light_travel_time(target, 'heliocentric', location=location)
    
    # Apply correction to get HJD
    hjd = time.tdb + light_time_correction
    return hjd.jd

def type_band(rr_type, band):
    if rr_type == 'RRab':
        pulsation_type = 0
    else:
        pulsation_type = 1

    if band == 'g':
        passband = 2
    elif band == 'r':
        passband = 3
    elif band == 'i':
        passband = 4

    if pulsation_type == 0:
        amplmaxs = {2: 1.8, 3: 1.5, 4: 1.1, 5: 1.1}
        amplmins = {2: .2, 3: .15, 4: .1, 5: .1}
        amplratio_ogle_to_use = 'Ampl(x)/Ampl(I)_ab'
    else:
        amplmaxs = {2: .9, 3: .7, 4: .55, 5: 1.1}
        amplmins = {2: .1, 3: .05, 4: 0., 5: 0.}    
        amplratio_ogle_to_use = 'Ampl(x)/Ampl(I)_c'

    amplmax = amplmaxs[passband]
    amplmin = amplmins[passband]
        
    return pulsation_type, passband, amplmax, amplmin

def apply_light_curve_fit(data, file_coeff, t0=0):
    obj, pulsation_period, rr_type, ra, dec, band, feh, feh_err, EBV, EBV_err, mu, mu_err = data[['object_id', 'Period', 'Type', 'RA', 'DEC','filter',
                                                          "[Fe/H]", "[Fe/H]_error", "E(B-V)", "E(B-V)_error", 
                                                          "Distance_modulus", "Distance_modulus_error"]].iloc[0]
    pulsation_type, passband, ampl_max, ampl_min = type_band(rr_type, band)
    
    mag  = np.asarray(data['mag_psf'])
    err  = np.asarray(data['e_mag_psf'])
    date = np.asarray(data['date'])

    hjd = mjd_to_hjd(date, ra, dec)
    try:
        templatebin_int = apply_ugrizy_templates.find_templatebin(pulsation_type, 
                                                                  pulsation_period, 
                                                                  passband)[0]
    
        result_resample = apply_ugrizy_templates.apply_templatefit(hjd, mag, err,
                                                    pulsation_type, pulsation_period, t0, passband, 
                                                    file_coeff, ampl=(ampl_max+ampl_min)/2,
                                                    amplmax=ampl_max, amplmin=ampl_min, figure_out=f'{folder}/plot-lcfit/{obj}-{band}.pdf')
        return {
            "object_id": obj,
            "period": pulsation_period,
            "type": rr_type,
            "band": band,
            "mean_mag": result_resample['mag_mean'],
            "err_mag": result_resample['errmag_mean'],
            "chi_squared": result_resample['chisq'],
            "[Fe/H]": feh,
            "[Fe/H]_error": feh_err,
            "E(B-V)": EBV,
            "E(B-V)_error": EBV_err,
            "Distance_modulus": mu,
            "Distance_modulus_error": mu_err,
        }
        
    except Exception as e:
        return None

In [6]:
objid = np.unique(merged_df['object_id'])[:100]

sample_dict = {f: {"good": [], "bad" : []} for f in ["g", "r", "i"]}

for obj in objid:
    current_target = merged_df[merged_df['object_id'] == obj]

    # Process all filters in one loop
    for band in ["g", "r", "i"]:
        image_data = current_target[current_target['filter'] == band]
        
        # Classify sample into good or bad
        category = "good" if mag_filter(image_data) else "bad"
        sample_dict[band][category].append(image_data)

In [7]:
folder = os.getcwd()+'/' #To be changed by the user
file_coeff = folder+'templates_analytics_230901.csv'

In [8]:
# Prepare the list of tasks
tasks = []
for band in ["g", "r", "i"]:
    tasks.extend([(data, file_coeff) for data in sample_dict[band]["good"]])
    
start = timeit.default_timer()

# Use multiprocessing to run tasks in parallel
if __name__ == "__main__":
    with mp.Pool(processes=mp.cpu_count()) as pool:  # Use all available CPU cores
        results_list = pool.starmap(apply_light_curve_fit, tasks)

    # Filter out None values (errors)
    results_list = [res for res in results_list if res is not None]

    # Convert results to a DataFrame and save to CSV
    df_results = pd.DataFrame(results_list)
    df_results.to_csv("light_curve_results.csv", index=False, float_format="%.6f")

    print("CSV export complete: light_curve_results.csv")

stop = timeit.default_timer()

print('Fitting time:', stop - start)

CSV export complete: light_curve_results.csv
Fitting time: 5.600935022000158


## Plot

In [3]:
df = pd.read_csv("light_curve_results.csv")

In [4]:
mg = (df['band'] == 'g') & ((df['chi_squared'] < 10) | (df['err_mag'] < 1))
mr = (df['band'] == 'r') & ((df['chi_squared'] < 10) | (df['err_mag'] < 1))
mi = (df['band'] == 'i') & ((df['chi_squared'] < 10) | (df['err_mag'] < 1))

df_g = df[mg]
df_r = df[mr]
df_i = df[mi]